In [1]:
import argparse
import os
import os.path as osp

import numpy as np
import torch
print(torch.cuda.is_available())

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import network
import loss
import pre_process as prep
import lr_schedule
from pre_process import ImageList, image_classification_test
import copy
import random



if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Code for RSDA-MSTN')
    parser.add_argument('--gpu_id', type=str, nargs='?', default='0', help="device id to run")
    parser.add_argument('--source', type=str, default='amazon',choices=["amazon", "dslr","webcam"])
    parser.add_argument('--target', type=str, default='dslr', choices=["amazon", "dslr", "webcam"])
    parser.add_argument('--test_interval', type=int, default=50, help="interval of two continuous test phase")
    parser.add_argument('--snapshot_interval', type=int, default=1000, help="interval of two continuous output model")
    parser.add_argument('--lr', type=float, default=0.001, help="learning rate")
    parser.add_argument('--stages', type=int, default=6, help="training stages")
    args = parser.parse_args([])
    s_dset_path = 'data/office/' + args.source + '_list.txt' #'../../data/office/' + args.source + '_list.txt'
    t_dset_path = 'data/office/' + args.target + '_list.txt' #'../../data/office/' + args.target + '_list.txt'

    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
    config = {}
    config["source"] = args.source
    config["target"] = args.target
    config["gpu"] = args.gpu_id
    config["test_interval"] = args.test_interval
    config["snapshot_interval"] = args.snapshot_interval
    config["output_for_test"] = True
    config["output_path"] = "snapshot/init"
    if not osp.exists(config["output_path"]):
        os.makedirs(config["output_path"])
    config["out_file"] = open(osp.join(config["output_path"],args.source+"_"+args.target+ "no_sm_log.txt"), "w")

    config["prep"] = {'params':{"resize_size":256, "crop_size":224}}
    config["network"] = {"name":network.ResNet50, \
            "params":{"new_cls":True,"feature_dim":256,"class_num":31} }
    config["optimizer"] = {"type":optim.SGD, "optim_params":{'lr':args.lr, "momentum":0.9, \
                           "weight_decay":0.0005, "nesterov":True}, "lr_type":"inv", \
                           "lr_param":{"lr":args.lr, "gamma":0.001, "power":0.75} }
    config["data"] = {"source":{"list_path":s_dset_path, "batch_size":36}, \
                      "target":{"list_path":t_dset_path, "batch_size":36}, \
                      "test":{"list_path":t_dset_path, "batch_size":72}}
    config["out_file"].flush()
    if config["source"] == "amazon" and config["target"] == "dslr":
        config["iterations"] = 2000
        seed = 0
    elif config["source"] == "amazon" and config["target"] == "webcam":
        config["iterations"] = 2000
        seed = 0
    else:
        config["iterations"] = 4000
        seed = 1


    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

    config["out_file"].write('\n--- initialization ---\n')
    source = config["source"]
    target = config["target"]
    prep_dict = {}
    prep_dict["source"] = prep.image_train(**config["prep"]['params'])
    prep_dict["target"] = prep.image_train(**config["prep"]['params'])

    prep_dict["test"] = prep.image_test(**config["prep"]['params'])

    ## prepare data
    dsets = {}
    dset_loaders = {}
    data_config = config["data"]
    train_bs = data_config["source"]["batch_size"]
    test_bs = data_config["test"]["batch_size"]
    dsets["source"] = ImageList(open(data_config["source"]["list_path"]).readlines(), \
                                transform=prep_dict["source"])
    dset_loaders["source"] = DataLoader(dsets["source"], batch_size=train_bs, \
                                        shuffle=True, num_workers=0, drop_last=True)
    dsets["target"] = ImageList(open(data_config["target"]["list_path"]).readlines(), \
                                transform=prep_dict["target"])
    dset_loaders["target"] = DataLoader(dsets["target"], batch_size=train_bs, \
                                        shuffle=True, num_workers=0, drop_last=True)

    dsets["test"] = ImageList(open(data_config["test"]["list_path"]).readlines(), \
                              transform=prep_dict["test"])
    dset_loaders["test"] = DataLoader(dsets["test"], batch_size=test_bs, \
                                      shuffle=False, num_workers=0)

    class_num = config["network"]["params"]["class_num"]

    ## set base network
    net_config = config["network"]
    base_network = net_config["name"](**net_config["params"])
    base_network = base_network.cuda()

    ## add additional network for some methods
    ad_net = network.AdversarialNetwork(base_network.output_num(), 1024)
    ad_net = ad_net.cuda()

    gpus = config['gpu'].split(',')
    if len(gpus) > 1:
        ad_net = nn.DataParallel(ad_net)
        base_network = nn.DataParallel(base_network)

    parameter_classifier = [base_network.get_parameters()[1]]
    parameter_feature = base_network.get_parameters()[0:1] + ad_net.get_parameters()

    ## set optimizer
    optimizer_config = config["optimizer"]
    optimizer_classfier = optimizer_config["type"](parameter_classifier, \
                                                   **(optimizer_config["optim_params"]))
    optimizer_feature = optimizer_config["type"](parameter_feature, \
                                                 **(optimizer_config["optim_params"]))
    param_lr = []
    for param_group in optimizer_feature.param_groups:
        param_lr.append(param_group["lr"])
    param_lr.append(optimizer_classfier.param_groups[0]["lr"])
    schedule_param = optimizer_config["lr_param"]
    lr_scheduler = lr_schedule.schedule_dict[optimizer_config["lr_type"]]

    ## train
    len_train_source = len(dset_loaders["source"])
    len_train_target = len(dset_loaders["target"])
    best_acc = 0.0
    best_model = copy.deepcopy(base_network)

    Cs_memory=torch.zeros(class_num,256).cuda()
    Ct_memory=torch.zeros(class_num,256).cuda()


    for i in range(config["iterations"]):
        if i % config["test_interval"] == config["test_interval"] - 1:
            base_network.train(False)
            temp_acc = image_classification_test(dset_loaders,base_network)
            temp_model = base_network
            if temp_acc > best_acc:
                best_acc = temp_acc
                best_model = copy.deepcopy(temp_model)
            log_str = "iter: {:05d}, \t precision: {:.4f},\t best_acc:{:.4f}".format(i, temp_acc, best_acc)
            config["out_file"].write(log_str + "\n")
            config["out_file"].flush()
            print(log_str)
        if (i + 1) % config["snapshot_interval"] == 0:
            if not os.path.exists("save/init_model"):
                os.makedirs("save/init_model")
            torch.save(best_model, 'save/init_model/' + source + '_' + target + 'no_sm.pkl')

        ## train one iter
        base_network.train(True)
        ad_net.train(True)
        optimizer_classfier = lr_scheduler(optimizer_classfier, i, **schedule_param)
        optimizer_feature = lr_scheduler(optimizer_feature, i, **schedule_param)

        if i % len_train_source == 0:
            iter_source = iter(dset_loaders["source"])
        if i % len_train_target == 0:
            iter_target = iter(dset_loaders["target"])
        inputs_source, labels_source = iter_source.next()
        inputs_target, labels_target = iter_target.next()
        inputs_source, inputs_target, labels_source = inputs_source.cuda(), inputs_target.cuda(), labels_source.cuda()
        features_source, outputs_source = base_network(inputs_source)
        features_target, outputs_target = base_network(inputs_target)
        features = torch.cat((features_source, features_target), dim=0)
        transfer_loss = loss.DANN(features, ad_net)
        pseu_labels_target=torch.argmax(outputs_target,dim=1)
        
        classifier_loss = nn.CrossEntropyLoss()(outputs_source, labels_source)

        loss_total = classifier_loss + transfer_loss

        optimizer_classfier.zero_grad()
        optimizer_feature.zero_grad()

        loss_total.backward()
        optimizer_feature.step()
        optimizer_classfier.step()

        print('step:{: d},\t,class_loss:{:.4f},\t,trans_loss:{:.4f},\t,'
              ''.format(i, classifier_loss.item(),transfer_loss.item()))
        Cs_memory.detach_()
        Ct_memory.detach_()

True
step: 0,	,class_loss:3.6366,	,trans_loss:0.6927,	,
step: 1,	,class_loss:3.4498,	,trans_loss:0.6928,	,
step: 2,	,class_loss:3.4750,	,trans_loss:0.6931,	,
step: 3,	,class_loss:3.5438,	,trans_loss:0.6930,	,
step: 4,	,class_loss:3.4940,	,trans_loss:0.6935,	,
step: 5,	,class_loss:3.5317,	,trans_loss:0.6926,	,
step: 6,	,class_loss:3.4987,	,trans_loss:0.6932,	,
step: 7,	,class_loss:3.5109,	,trans_loss:0.6927,	,
step: 8,	,class_loss:3.3171,	,trans_loss:0.6926,	,
step: 9,	,class_loss:3.3008,	,trans_loss:0.6935,	,
step: 10,	,class_loss:3.3447,	,trans_loss:0.6933,	,
step: 11,	,class_loss:3.3857,	,trans_loss:0.6936,	,
step: 12,	,class_loss:3.3408,	,trans_loss:0.6921,	,
step: 13,	,class_loss:3.3395,	,trans_loss:0.6930,	,
step: 14,	,class_loss:3.2675,	,trans_loss:0.6931,	,
step: 15,	,class_loss:3.2586,	,trans_loss:0.6919,	,
step: 16,	,class_loss:3.2178,	,trans_loss:0.6926,	,
step: 17,	,class_loss:3.2330,	,trans_loss:0.6924,	,
step: 18,	,class_loss:3.2146,	,trans_loss:0.6929,	,
step: 19,	,class_

step: 154,	,class_loss:1.2641,	,trans_loss:0.6903,	,
step: 155,	,class_loss:0.7970,	,trans_loss:0.6896,	,
step: 156,	,class_loss:0.8155,	,trans_loss:0.6886,	,
step: 157,	,class_loss:0.7666,	,trans_loss:0.6888,	,
step: 158,	,class_loss:0.7612,	,trans_loss:0.6889,	,
step: 159,	,class_loss:0.7472,	,trans_loss:0.6891,	,
step: 160,	,class_loss:1.0315,	,trans_loss:0.6875,	,
step: 161,	,class_loss:0.9664,	,trans_loss:0.6880,	,
step: 162,	,class_loss:1.0156,	,trans_loss:0.6876,	,
step: 163,	,class_loss:0.5882,	,trans_loss:0.6884,	,
step: 164,	,class_loss:0.7279,	,trans_loss:0.6879,	,
step: 165,	,class_loss:0.5971,	,trans_loss:0.6890,	,
step: 166,	,class_loss:0.8231,	,trans_loss:0.6885,	,
step: 167,	,class_loss:1.0441,	,trans_loss:0.6874,	,
step: 168,	,class_loss:0.8790,	,trans_loss:0.6880,	,
step: 169,	,class_loss:0.8715,	,trans_loss:0.6879,	,
step: 170,	,class_loss:1.3688,	,trans_loss:0.6880,	,
step: 171,	,class_loss:1.0203,	,trans_loss:0.6892,	,
step: 172,	,class_loss:0.6604,	,trans_loss:0.6

step: 306,	,class_loss:0.4580,	,trans_loss:0.6846,	,
step: 307,	,class_loss:0.5825,	,trans_loss:0.6886,	,
step: 308,	,class_loss:0.7315,	,trans_loss:0.6848,	,
step: 309,	,class_loss:0.6013,	,trans_loss:0.6822,	,
step: 310,	,class_loss:0.5923,	,trans_loss:0.6854,	,
step: 311,	,class_loss:0.4552,	,trans_loss:0.6838,	,
step: 312,	,class_loss:0.3402,	,trans_loss:0.6840,	,
step: 313,	,class_loss:0.4276,	,trans_loss:0.6836,	,
step: 314,	,class_loss:0.5576,	,trans_loss:0.6838,	,
step: 315,	,class_loss:0.5055,	,trans_loss:0.6845,	,
step: 316,	,class_loss:0.5988,	,trans_loss:0.6856,	,
step: 317,	,class_loss:0.4296,	,trans_loss:0.6841,	,
step: 318,	,class_loss:0.4341,	,trans_loss:0.6852,	,
step: 319,	,class_loss:0.6516,	,trans_loss:0.6848,	,
step: 320,	,class_loss:0.4520,	,trans_loss:0.6852,	,
step: 321,	,class_loss:0.3739,	,trans_loss:0.6882,	,
step: 322,	,class_loss:0.4272,	,trans_loss:0.6842,	,
step: 323,	,class_loss:0.3843,	,trans_loss:0.6815,	,
step: 324,	,class_loss:0.3982,	,trans_loss:0.6

step: 458,	,class_loss:0.4821,	,trans_loss:0.6848,	,
step: 459,	,class_loss:0.5823,	,trans_loss:0.6817,	,
step: 460,	,class_loss:0.3265,	,trans_loss:0.6804,	,
step: 461,	,class_loss:0.4259,	,trans_loss:0.6836,	,
step: 462,	,class_loss:0.2929,	,trans_loss:0.6812,	,
step: 463,	,class_loss:0.4280,	,trans_loss:0.6776,	,
step: 464,	,class_loss:0.3839,	,trans_loss:0.6829,	,
step: 465,	,class_loss:0.2496,	,trans_loss:0.6827,	,
step: 466,	,class_loss:0.1623,	,trans_loss:0.6839,	,
step: 467,	,class_loss:0.3737,	,trans_loss:0.6847,	,
step: 468,	,class_loss:0.2115,	,trans_loss:0.6842,	,
step: 469,	,class_loss:0.3623,	,trans_loss:0.6833,	,
step: 470,	,class_loss:0.3227,	,trans_loss:0.6830,	,
step: 471,	,class_loss:0.1473,	,trans_loss:0.6809,	,
step: 472,	,class_loss:0.1929,	,trans_loss:0.6809,	,
step: 473,	,class_loss:0.2743,	,trans_loss:0.6836,	,
step: 474,	,class_loss:0.3353,	,trans_loss:0.6793,	,
step: 475,	,class_loss:0.3787,	,trans_loss:0.6794,	,
step: 476,	,class_loss:0.1779,	,trans_loss:0.6

step: 610,	,class_loss:0.3195,	,trans_loss:0.6847,	,
step: 611,	,class_loss:0.3579,	,trans_loss:0.6879,	,
step: 612,	,class_loss:0.2633,	,trans_loss:0.6862,	,
step: 613,	,class_loss:0.1868,	,trans_loss:0.6866,	,
step: 614,	,class_loss:0.2129,	,trans_loss:0.6814,	,
step: 615,	,class_loss:0.2311,	,trans_loss:0.6850,	,
step: 616,	,class_loss:0.1357,	,trans_loss:0.6876,	,
step: 617,	,class_loss:0.1322,	,trans_loss:0.6857,	,
step: 618,	,class_loss:0.1795,	,trans_loss:0.6859,	,
step: 619,	,class_loss:0.1876,	,trans_loss:0.6850,	,
step: 620,	,class_loss:0.2277,	,trans_loss:0.6868,	,
step: 621,	,class_loss:0.1429,	,trans_loss:0.6891,	,
step: 622,	,class_loss:0.4236,	,trans_loss:0.6884,	,
step: 623,	,class_loss:0.2106,	,trans_loss:0.6855,	,
step: 624,	,class_loss:0.2815,	,trans_loss:0.6826,	,
step: 625,	,class_loss:0.2883,	,trans_loss:0.6884,	,
step: 626,	,class_loss:0.1626,	,trans_loss:0.6855,	,
step: 627,	,class_loss:0.2542,	,trans_loss:0.6913,	,
step: 628,	,class_loss:0.2167,	,trans_loss:0.6

...

step: 1363,	,class_loss:0.0424,	,trans_loss:0.6965,	,
step: 1364,	,class_loss:0.0602,	,trans_loss:0.6962,	,
step: 1365,	,class_loss:0.0362,	,trans_loss:0.6956,	,
step: 1366,	,class_loss:0.0355,	,trans_loss:0.6968,	,
step: 1367,	,class_loss:0.0974,	,trans_loss:0.6941,	,
step: 1368,	,class_loss:0.0855,	,trans_loss:0.6950,	,
step: 1369,	,class_loss:0.0357,	,trans_loss:0.6951,	,
step: 1370,	,class_loss:0.0422,	,trans_loss:0.6950,	,
step: 1371,	,class_loss:0.0450,	,trans_loss:0.6968,	,
step: 1372,	,class_loss:0.0915,	,trans_loss:0.6944,	,
step: 1373,	,class_loss:0.0326,	,trans_loss:0.6956,	,
step: 1374,	,class_loss:0.0303,	,trans_loss:0.6952,	,
step: 1375,	,class_loss:0.0544,	,trans_loss:0.6953,	,
step: 1376,	,class_loss:0.0566,	,trans_loss:0.6945,	,
step: 1377,	,class_loss:0.0500,	,trans_loss:0.6957,	,
step: 1378,	,class_loss:0.0264,	,trans_loss:0.6959,	,
step: 1379,	,class_loss:0.0333,	,trans_loss:0.6953,	,
step: 1380,	,class_loss:0.0932,	,trans_loss:0.6946,	,
step: 1381,	,class_loss:0.04

step: 1512,	,class_loss:0.0316,	,trans_loss:0.6951,	,
step: 1513,	,class_loss:0.0392,	,trans_loss:0.6939,	,
step: 1514,	,class_loss:0.0743,	,trans_loss:0.6941,	,
step: 1515,	,class_loss:0.0563,	,trans_loss:0.6957,	,
step: 1516,	,class_loss:0.0329,	,trans_loss:0.6927,	,
step: 1517,	,class_loss:0.0552,	,trans_loss:0.6950,	,
step: 1518,	,class_loss:0.0882,	,trans_loss:0.6936,	,
step: 1519,	,class_loss:0.0790,	,trans_loss:0.6943,	,
step: 1520,	,class_loss:0.0918,	,trans_loss:0.6933,	,
step: 1521,	,class_loss:0.0701,	,trans_loss:0.6938,	,
step: 1522,	,class_loss:0.0366,	,trans_loss:0.6947,	,
step: 1523,	,class_loss:0.0434,	,trans_loss:0.6943,	,
step: 1524,	,class_loss:0.0335,	,trans_loss:0.6945,	,
step: 1525,	,class_loss:0.0271,	,trans_loss:0.6944,	,
step: 1526,	,class_loss:0.0578,	,trans_loss:0.6946,	,
step: 1527,	,class_loss:0.0755,	,trans_loss:0.6942,	,
step: 1528,	,class_loss:0.0304,	,trans_loss:0.6944,	,
step: 1529,	,class_loss:0.0416,	,trans_loss:0.6963,	,
step: 1530,	,class_loss:0.03

step: 1661,	,class_loss:0.0390,	,trans_loss:0.6935,	,
step: 1662,	,class_loss:0.0247,	,trans_loss:0.6937,	,
step: 1663,	,class_loss:0.0612,	,trans_loss:0.6939,	,
step: 1664,	,class_loss:0.0269,	,trans_loss:0.6941,	,
step: 1665,	,class_loss:0.0797,	,trans_loss:0.6943,	,
step: 1666,	,class_loss:0.0283,	,trans_loss:0.6935,	,
step: 1667,	,class_loss:0.0650,	,trans_loss:0.6940,	,
step: 1668,	,class_loss:0.0826,	,trans_loss:0.6935,	,
step: 1669,	,class_loss:0.0533,	,trans_loss:0.6940,	,
step: 1670,	,class_loss:0.0570,	,trans_loss:0.6931,	,
step: 1671,	,class_loss:0.0199,	,trans_loss:0.6927,	,
step: 1672,	,class_loss:0.0137,	,trans_loss:0.6921,	,
step: 1673,	,class_loss:0.0462,	,trans_loss:0.6933,	,
step: 1674,	,class_loss:0.0477,	,trans_loss:0.6941,	,
step: 1675,	,class_loss:0.0524,	,trans_loss:0.6933,	,
step: 1676,	,class_loss:0.0406,	,trans_loss:0.6941,	,
step: 1677,	,class_loss:0.0309,	,trans_loss:0.6922,	,
step: 1678,	,class_loss:0.0246,	,trans_loss:0.6945,	,
step: 1679,	,class_loss:0.07

step: 1810,	,class_loss:0.0238,	,trans_loss:0.6943,	,
step: 1811,	,class_loss:0.0280,	,trans_loss:0.6916,	,
step: 1812,	,class_loss:0.0992,	,trans_loss:0.6933,	,
step: 1813,	,class_loss:0.0692,	,trans_loss:0.6925,	,
step: 1814,	,class_loss:0.0389,	,trans_loss:0.6944,	,
step: 1815,	,class_loss:0.0128,	,trans_loss:0.6936,	,
step: 1816,	,class_loss:0.0188,	,trans_loss:0.6930,	,
step: 1817,	,class_loss:0.0449,	,trans_loss:0.6930,	,
step: 1818,	,class_loss:0.0475,	,trans_loss:0.6929,	,
step: 1819,	,class_loss:0.0364,	,trans_loss:0.6927,	,
step: 1820,	,class_loss:0.0374,	,trans_loss:0.6935,	,
step: 1821,	,class_loss:0.0514,	,trans_loss:0.6927,	,
step: 1822,	,class_loss:0.0356,	,trans_loss:0.6933,	,
step: 1823,	,class_loss:0.0349,	,trans_loss:0.6935,	,
step: 1824,	,class_loss:0.0291,	,trans_loss:0.6918,	,
step: 1825,	,class_loss:0.0174,	,trans_loss:0.6943,	,
step: 1826,	,class_loss:0.0215,	,trans_loss:0.6937,	,
step: 1827,	,class_loss:0.1566,	,trans_loss:0.6933,	,
step: 1828,	,class_loss:0.04

step: 1959,	,class_loss:0.0203,	,trans_loss:0.6928,	,
step: 1960,	,class_loss:0.0645,	,trans_loss:0.6937,	,
step: 1961,	,class_loss:0.0206,	,trans_loss:0.6941,	,
step: 1962,	,class_loss:0.0177,	,trans_loss:0.6930,	,
step: 1963,	,class_loss:0.0231,	,trans_loss:0.6939,	,
step: 1964,	,class_loss:0.0378,	,trans_loss:0.6935,	,
step: 1965,	,class_loss:0.0430,	,trans_loss:0.6929,	,
step: 1966,	,class_loss:0.0243,	,trans_loss:0.6941,	,
step: 1967,	,class_loss:0.0347,	,trans_loss:0.6917,	,
step: 1968,	,class_loss:0.0832,	,trans_loss:0.6942,	,
step: 1969,	,class_loss:0.0373,	,trans_loss:0.6940,	,
step: 1970,	,class_loss:0.0234,	,trans_loss:0.6926,	,
step: 1971,	,class_loss:0.0128,	,trans_loss:0.6943,	,
step: 1972,	,class_loss:0.0227,	,trans_loss:0.6919,	,
step: 1973,	,class_loss:0.0207,	,trans_loss:0.6927,	,
step: 1974,	,class_loss:0.0148,	,trans_loss:0.6944,	,
step: 1975,	,class_loss:0.0324,	,trans_loss:0.6930,	,
step: 1976,	,class_loss:0.0714,	,trans_loss:0.6937,	,
step: 1977,	,class_loss:0.01